In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
import keras
from keras import Model, Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Input
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
def load_data(nb_train=40000, nb_test=10000, full=False):
    """
    Function to load data in the keras way.
    
    Parameters
    ----------
    nb_train (int): Number of training examples
    nb_test (int): Number of testing examples
    full (bool): If True, whole csv will be loaded, nb_test will be ignored
    
    Returns
    -------
    Xtrain, ytrain (np.array, np.array),
        shapes (nb_train, 9, 9), (nb_train, 9, 9): Training samples
    Xtest, ytest (np.array, np.array), 
        shapes (nb_test, 9, 9), (nb_test, 9, 9): Testing samples
    """
    # if full is true, load the whole dataset
    if full:
        sudokus = pd.read_csv('sudoku.csv').values
    else:
        sudokus = next(
            pd.read_csv('sudoku.csv', chunksize=(nb_train + nb_test))
        ).values
        quizzes, solutions = sudokus.T
    flatX = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in quizzes])
    flaty = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in solutions])
    
    return (flatX[:nb_train], flaty[:nb_train]), (flatX[nb_train:], flaty[nb_train:])


def diff(grids_true, grids_pred):
    """
    This function shows how well predicted quizzes fit to actual solutions.
    It will store sum of differences for each pair (solution, guess)
    
    Parameters
    ----------
    grids_true (np.array), shape (?, 9, 9): Real solutions to guess in the digit repesentation
    grids_pred (np.array), shape (?, 9, 9): Guesses
    
    Returns
    -------
    diff (np.array), shape (?,): Number of differences for each pair (solution, guess)
    """
    return (grids_true != grids_pred).sum((1, 2))
def delete_digits(X, n_delet=1):
    """
    This function is used to create sudoku quizzes from solutions
    
    Parameters
    ----------
    X (np.array), shape (?, 9, 9, 9|10): input solutions grids.
    n_delet (integer): max number of digit to suppress from original solutions
    
    Returns
    -------
    grids: np.array of grids to guess in one-hot way. Shape: (?, 9, 9, 10)
    """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, n_delet)] = 0  # generate blanks (replace = True)
    return to_categorical(grids)


def batch_smart_solve(grids, solver):
    """
    NOTE : This function is ugly, feel free to optimize the code ...
    
    This function solves quizzes in the "smart" way. 
    It will fill blanks one after the other. Each time a digit is filled, 
    the new grid will be fed again to the solver to predict the next digit. 
    Again and again, until there is no more blank
    
    Parameters
    ----------
    grids (np.array), shape (?, 9, 9): Batch of quizzes to solve (smartly ;))
    solver (keras.model): The neural net solver
    
    Returns
    -------grids (np.array), shape (?, 9, 9): Smartly solved quizzes.
    """
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        preds = np.array(solver.predict(to_categorical(grids)))  # get predictions
        probs = preds.max(2).T  # get highest probability for each 81 digit to predict
        values = preds.argmax(2).T + 1  # get corresponding values
        zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            if any(zero):  # don't try to fill already completed grid
                where = np.where(zero)[0]  # focus on blanks only
                confidence_position = where[prob[zero].argmax()]  # best score FOR A ZERO VALUE (confident blank)
                confidence_value = value[confidence_position]  # get corresponding value
                grid.flat[confidence_position] = confidence_value  # fill digit inplace
    return grids

In [4]:
input_shape = (9, 9, 10)
(_, ytrain), (Xtest, ytest) = load_data()  # We won't use _. We will work directly with ytrain

# one-hot-encoding --> shapes become :
# (?, 9, 9, 10) for Xs
# (?, 9, 9, 9) for ys
Xtrain = to_categorical(ytrain).astype('float32')  # from ytrain cause we will creates quizzes from solusions
Xtest = to_categorical(Xtest).astype('float32')

ytrain = to_categorical(ytrain-1).astype('float32') # (y - 1) because we 
ytest = to_categorical(ytest-1).astype('float32')   # don't want to predict zeros

In [7]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=input_shape))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())

grid = Input(shape=input_shape)  # inputs
features = model(grid)  # commons features
# define one Dense layer for each of the digit we want to predict
digit_placeholders = [
    Dense(9, activation='softmax')(features)
    for i in range(81)
]

solver = Model(grid, digit_placeholders)  # build the whole model
solver.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
solver.fit(
    delete_digits(Xtrain, 0),  # we don't delete any digit for now
    [ytrain[:, i, j, :] for i in range(9) for j in range(9)],  # each digit of solution
    batch_size=128,
    epochs=1,  # 1 epoch should be enough for the task
    verbose=1,
)


Epoch 1/1
40000/40000 [==============================] - 142s 4ms/step - loss: 44.3305 - dense_7_loss: 0.5421 - dense_8_loss: 0.5513 - dense_9_loss: 0.5417 - dense_10_loss: 0.5458 - dense_11_loss: 0.5500 - dense_12_loss: 0.5487 - dense_13_loss: 0.5453 - dense_14_loss: 0.5486 - dense_15_loss: 0.5449 - dense_16_loss: 0.5426 - dense_17_loss: 0.5473 - dense_18_loss: 0.5458 - dense_19_loss: 0.5450 - dense_20_loss: 0.5439 - dense_21_loss: 0.5484 - dense_22_loss: 0.5439 - dense_23_loss: 0.5454 - dense_24_loss: 0.5451 - dense_25_loss: 0.5455 - dense_26_loss: 0.5453 - dense_27_loss: 0.5429 - dense_28_loss: 0.5504 - dense_29_loss: 0.5496 - dense_30_loss: 0.5434 - dense_31_loss: 0.5467 - dense_32_loss: 0.5458 - dense_33_loss: 0.5448 - dense_34_loss: 0.5507 - dense_35_loss: 0.5433 - dense_36_loss: 0.5400 - dense_37_loss: 0.5497 - dense_38_loss: 0.5440 - dense_39_loss: 0.5524 - dense_40_loss: 0.5403 - dense_41_loss: 0.5536 - dense_42_loss: 0.5444 - dense_43_loss: 0.5474 - dense_44_loss: 0.5422 - d

In [9]:
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1

for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    solver.fit(
        delete_digits(Xtrain, nb_delete),  # delete digits from training sample
        [ytrain[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(Xtrain, nb_delete), # delete same amount of digit from validation sample
            [ytrain[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=128,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )

Pass n° 1 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/1
40000/40000 [==============================] - 161s 4ms/step - loss: 1.2089 - dense_7_loss: 0.0156 - dense_8_loss: 0.0142 - dense_9_loss: 0.0169 - dense_10_loss: 0.0152 - dense_11_loss: 0.0150 - dense_12_loss: 0.0164 - dense_13_loss: 0.0149 - dense_14_loss: 0.0166 - dense_15_loss: 0.0165 - dense_16_loss: 0.0141 - dense_17_loss: 0.0146 - dense_18_loss: 0.0150 - dense_19_loss: 0.0144 - dense_20_loss: 0.0154 - dense_21_loss: 0.0154 - dense_22_loss: 0.0149 - dense_23_loss: 0.0135 - dense_24_loss: 0.0155 - dense_25_loss: 0.0153 - dense_26_loss: 0.0139 - dense_27_loss: 0.0150 - dense_28_loss: 0.0148 - dense_29_loss: 0.0160 - dense_30_loss: 0.0140 - dense_31_loss: 0.0160 - dense_32_loss: 0.0149 - dense_33_loss: 0.0158 - dense_34_loss: 0.0139 - dense_35_loss: 0.0164 - dense_36_loss: 0.0151 - dense_37_loss: 0.0156 - dense_38_loss: 0.0154 - dense_39_loss: 0.0138 - dense_40_loss: 0.0129 - dense_41_loss: 0.0149 - dense_42_lo

Pass n° 2 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/2
40000/40000 [==============================] - 157s 4ms/step - loss: 1.4446 - dense_7_loss: 0.0193 - dense_8_loss: 0.0188 - dense_9_loss: 0.0206 - dense_10_loss: 0.0179 - dense_11_loss: 0.0184 - dense_12_loss: 0.0156 - dense_13_loss: 0.0183 - dense_14_loss: 0.0179 - dense_15_loss: 0.0163 - dense_16_loss: 0.0192 - dense_17_loss: 0.0183 - dense_18_loss: 0.0190 - dense_19_loss: 0.0200 - dense_20_loss: 0.0171 - dense_21_loss: 0.0184 - dense_22_loss: 0.0204 - dense_23_loss: 0.0195 - dense_24_loss: 0.0178 - dense_25_loss: 0.0195 - dense_26_loss: 0.0200 - dense_27_loss: 0.0192 - dense_28_loss: 0.0155 - dense_29_loss: 0.0162 - dense_30_loss: 0.0180 - dense_31_loss: 0.0160 - dense_32_loss: 0.0172 - dense_33_loss: 0.0185 - dense_34_loss: 0.0181 - dense_35_loss: 0.0166 - dense_36_loss: 0.0173 - dense_37_loss: 0.0167 - dense_38_loss: 0.0211 - dense_39_loss: 0.0170 - dense_40_loss: 0.0172 - dense_41_loss: 0.0188 - dense_42_lo

Epoch 2/2
40000/40000 [==============================] - 157s 4ms/step - loss: 1.0199 - dense_7_loss: 0.0119 - dense_8_loss: 0.0132 - dense_9_loss: 0.0121 - dense_10_loss: 0.0142 - dense_11_loss: 0.0160 - dense_12_loss: 0.0136 - dense_13_loss: 0.0117 - dense_14_loss: 0.0132 - dense_15_loss: 0.0127 - dense_16_loss: 0.0129 - dense_17_loss: 0.0137 - dense_18_loss: 0.0130 - dense_19_loss: 0.0140 - dense_20_loss: 0.0127 - dense_21_loss: 0.0113 - dense_22_loss: 0.0119 - dense_23_loss: 0.0154 - dense_24_loss: 0.0103 - dense_25_loss: 0.0131 - dense_26_loss: 0.0102 - dense_27_loss: 0.0116 - dense_28_loss: 0.0154 - dense_29_loss: 0.0128 - dense_30_loss: 0.0128 - dense_31_loss: 0.0119 - dense_32_loss: 0.0121 - dense_33_loss: 0.0115 - dense_34_loss: 0.0128 - dense_35_loss: 0.0130 - dense_36_loss: 0.0121 - dense_37_loss: 0.0120 - dense_38_loss: 0.0122 - dense_39_loss: 0.0123 - dense_40_loss: 0.0123 - dense_41_loss: 0.0134 - dense_42_loss: 0.0130 - dense_43_loss: 0.0104 - dense_44_loss: 0.0122 - den

Pass n° 3 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/3
40000/40000 [==============================] - 156s 4ms/step - loss: 1.7478 - dense_7_loss: 0.0207 - dense_8_loss: 0.0198 - dense_9_loss: 0.0210 - dense_10_loss: 0.0213 - dense_11_loss: 0.0226 - dense_12_loss: 0.0204 - dense_13_loss: 0.0213 - dense_14_loss: 0.0216 - dense_15_loss: 0.0214 - dense_16_loss: 0.0188 - dense_17_loss: 0.0241 - dense_18_loss: 0.0204 - dense_19_loss: 0.0204 - dense_20_loss: 0.0203 - dense_21_loss: 0.0212 - dense_22_loss: 0.0211 - dense_23_loss: 0.0212 - dense_24_loss: 0.0201 - dense_25_loss: 0.0220 - dense_26_loss: 0.0220 - dense_27_loss: 0.0191 - dense_28_loss: 0.0220 - dense_29_loss: 0.0241 - dense_30_loss: 0.0220 - dense_31_loss: 0.0225 - dense_32_loss: 0.0206 - dense_33_loss: 0.0219 - dense_34_loss: 0.0211 - dense_35_loss: 0.0210 - dense_36_loss: 0.0210 - dense_37_loss: 0.0257 - dense_38_loss: 0.0226 - dense_39_loss: 0.0208 - dense_40_loss: 0.0226 - dense_41_loss: 0.0236 - dense_42_lo

Epoch 2/3
40000/40000 [==============================] - 159s 4ms/step - loss: 1.0764 - dense_7_loss: 0.0143 - dense_8_loss: 0.0131 - dense_9_loss: 0.0136 - dense_10_loss: 0.0111 - dense_11_loss: 0.0130 - dense_12_loss: 0.0130 - dense_13_loss: 0.0122 - dense_14_loss: 0.0123 - dense_15_loss: 0.0130 - dense_16_loss: 0.0141 - dense_17_loss: 0.0135 - dense_18_loss: 0.0141 - dense_19_loss: 0.0120 - dense_20_loss: 0.0151 - dense_21_loss: 0.0141 - dense_22_loss: 0.0120 - dense_23_loss: 0.0122 - dense_24_loss: 0.0145 - dense_25_loss: 0.0123 - dense_26_loss: 0.0137 - dense_27_loss: 0.0129 - dense_28_loss: 0.0127 - dense_29_loss: 0.0124 - dense_30_loss: 0.0134 - dense_31_loss: 0.0138 - dense_32_loss: 0.0128 - dense_33_loss: 0.0121 - dense_34_loss: 0.0131 - dense_35_loss: 0.0147 - dense_36_loss: 0.0149 - dense_37_loss: 0.0141 - dense_38_loss: 0.0136 - dense_39_loss: 0.0124 - dense_40_loss: 0.0137 - dense_41_loss: 0.0115 - dense_42_loss: 0.0134 - dense_43_loss: 0.0141 - dense_44_loss: 0.0130 - den

...

Epoch 3/10
40000/40000 [==============================] - 127s 3ms/step - loss: 23.0882 - dense_7_loss: 0.2821 - dense_8_loss: 0.2811 - dense_9_loss: 0.2895 - dense_10_loss: 0.2825 - dense_11_loss: 0.2796 - dense_12_loss: 0.2903 - dense_13_loss: 0.2840 - dense_14_loss: 0.2825 - dense_15_loss: 0.2872 - dense_16_loss: 0.2840 - dense_17_loss: 0.2819 - dense_18_loss: 0.2822 - dense_19_loss: 0.2854 - dense_20_loss: 0.2809 - dense_21_loss: 0.2833 - dense_22_loss: 0.2823 - dense_23_loss: 0.2820 - dense_24_loss: 0.2823 - dense_25_loss: 0.2874 - dense_26_loss: 0.2894 - dense_27_loss: 0.2824 - dense_28_loss: 0.2880 - dense_29_loss: 0.2860 - dense_30_loss: 0.2875 - dense_31_loss: 0.2810 - dense_32_loss: 0.2819 - dense_33_loss: 0.2895 - dense_34_loss: 0.2849 - dense_35_loss: 0.2861 - dense_36_loss: 0.2889 - dense_37_loss: 0.2882 - dense_38_loss: 0.2901 - dense_39_loss: 0.2871 - dense_40_loss: 0.2873 - dense_41_loss: 0.2857 - dense_42_loss: 0.2855 - dense_43_loss: 0.2875 - dense_44_loss: 0.2852 - d

Epoch 00003: early stopping
Pass n° 17 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/10
40000/40000 [==============================] - 128s 3ms/step - loss: 32.0037 - dense_7_loss: 0.3914 - dense_8_loss: 0.3997 - dense_9_loss: 0.3929 - dense_10_loss: 0.3989 - dense_11_loss: 0.3939 - dense_12_loss: 0.3886 - dense_13_loss: 0.3938 - dense_14_loss: 0.3956 - dense_15_loss: 0.3957 - dense_16_loss: 0.3987 - dense_17_loss: 0.3931 - dense_18_loss: 0.3955 - dense_19_loss: 0.3948 - dense_20_loss: 0.3937 - dense_21_loss: 0.3929 - dense_22_loss: 0.3901 - dense_23_loss: 0.3963 - dense_24_loss: 0.3924 - dense_25_loss: 0.3970 - dense_26_loss: 0.3975 - dense_27_loss: 0.4007 - dense_28_loss: 0.3956 - dense_29_loss: 0.4002 - dense_30_loss: 0.3938 - dense_31_loss: 0.3951 - dense_32_loss: 0.3994 - dense_33_loss: 0.3996 - dense_34_loss: 0.3954 - dense_35_loss: 0.3915 - dense_36_loss: 0.3973 - dense_37_loss: 0.4021 - dense_38_loss: 0.3969 - dense_39_loss: 0.3946 - dense_40_loss: 0.3962 - dens

Epoch 2/10
40000/40000 [==============================] - 131s 3ms/step - loss: 29.2951 - dense_7_loss: 0.3575 - dense_8_loss: 0.3638 - dense_9_loss: 0.3605 - dense_10_loss: 0.3618 - dense_11_loss: 0.3640 - dense_12_loss: 0.3582 - dense_13_loss: 0.3643 - dense_14_loss: 0.3605 - dense_15_loss: 0.3585 - dense_16_loss: 0.3656 - dense_17_loss: 0.3592 - dense_18_loss: 0.3617 - dense_19_loss: 0.3617 - dense_20_loss: 0.3600 - dense_21_loss: 0.3607 - dense_22_loss: 0.3612 - dense_23_loss: 0.3625 - dense_24_loss: 0.3566 - dense_25_loss: 0.3648 - dense_26_loss: 0.3639 - dense_27_loss: 0.3667 - dense_28_loss: 0.3624 - dense_29_loss: 0.3660 - dense_30_loss: 0.3621 - dense_31_loss: 0.3578 - dense_32_loss: 0.3664 - dense_33_loss: 0.3656 - dense_34_loss: 0.3656 - dense_35_loss: 0.3588 - dense_36_loss: 0.3646 - dense_37_loss: 0.3689 - dense_38_loss: 0.3626 - dense_39_loss: 0.3599 - dense_40_loss: 0.3618 - dense_41_loss: 0.3642 - dense_42_loss: 0.3641 - dense_43_loss: 0.3578 - dense_44_loss: 0.3590 - d

Epoch 3/10
40000/40000 [==============================] - 133s 3ms/step - loss: 28.1460 - dense_7_loss: 0.3434 - dense_8_loss: 0.3502 - dense_9_loss: 0.3461 - dense_10_loss: 0.3490 - dense_11_loss: 0.3503 - dense_12_loss: 0.3449 - dense_13_loss: 0.3471 - dense_14_loss: 0.3461 - dense_15_loss: 0.3450 - dense_16_loss: 0.3496 - dense_17_loss: 0.3443 - dense_18_loss: 0.3476 - dense_19_loss: 0.3480 - dense_20_loss: 0.3439 - dense_21_loss: 0.3457 - dense_22_loss: 0.3464 - dense_23_loss: 0.3496 - dense_24_loss: 0.3431 - dense_25_loss: 0.3532 - dense_26_loss: 0.3493 - dense_27_loss: 0.3544 - dense_28_loss: 0.3476 - dense_29_loss: 0.3508 - dense_30_loss: 0.3484 - dense_31_loss: 0.3449 - dense_32_loss: 0.3495 - dense_33_loss: 0.3511 - dense_34_loss: 0.3524 - dense_35_loss: 0.3424 - dense_36_loss: 0.3513 - dense_37_loss: 0.3551 - dense_38_loss: 0.3484 - dense_39_loss: 0.3472 - dense_40_loss: 0.3445 - dense_41_loss: 0.3478 - dense_42_loss: 0.3489 - dense_43_loss: 0.3432 - dense_44_loss: 0.3474 - d

Epoch 00003: early stopping


In [10]:
quizzes = Xtest.argmax(3)  # quizzes in the (?, 9, 9) shape. From the test set
true_grids = ytest.argmax(3) + 1  # true solutions dont forget to add 1 
smart_guesses = batch_smart_solve(quizzes, solver)  # make smart guesses !

deltas = diff(true_grids, smart_guesses)  # get number of errors on each quizz
accuracy = (deltas == 0).mean()  # portion of correct solved quizzes

In [11]:
print(
"""
Grid solved:\t {}
Correct ones:\t {}
Accuracy:\t {}
""".format(
deltas.shape[0], (deltas==0).sum(), accuracy
)
)


Grid solved:	 10000
Correct ones:	 9953
Accuracy:	 0.9953

